In [35]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_score, silhouette_samples, davies_bouldin_score, calinski_harabasz_score, pairwise_distances
import datetime as dt

In [51]:
#Importamos los archivos de los clientes de electrodunas
df_c1 = pd.read_csv('./Datos Electro Dunas/DATOSCLIENTE1.csv')
#Observamos y agregamos a una base de datos los sectores económicos de cada cliente
sectores = pd.read_excel('./Datos Electro Dunas/sector_economico_clientes.xlsx')
#Agregamos la columna de cliente a la base de datos
df_c1['Cliente'] = 'Cliente 1'
# Renombrar la columna de la base de datos de sectores
sectores = sectores.rename(columns={'Cliente:':'Cliente'})
# Quitar los espacios en blanco de la columna de cliente
sectores['Cliente'] = sectores['Cliente'].str.strip()
#Unimos las bases de datos
df_c1 = df_c1.merge(sectores, on='Cliente', how='left')
# Dar formato de fecha a la columna de fecha
df_c1['Fecha'] = pd.to_datetime(df_c1['Fecha'])
# Crear una columna de mes
df_c1['Mes'] = df_c1['Fecha'].dt.month
# Crear una columna de año
df_c1['Año'] = df_c1['Fecha'].dt.year
# Crear una columna de día
df_c1['Dia'] = df_c1['Fecha'].dt.day
# Crear columna de hora
df_c1['Hora'] = df_c1['Fecha'].dt.hour
# Crear una columna de nombre del día
df_c1['Nombre_dia'] = df_c1['Fecha'].dt.day_name()
# Cambiar el formato de la fecha
df_c1['Fecha'] = df_c1['Fecha'].dt.strftime('%d/%m/%Y')
# Crear columna para fin de semana
df_c1['Fin_de_semana'] = df_c1['Nombre_dia'].apply(lambda x: 1 if x in ['Saturday', 'Sunday'] else 0)
# Crear columna para horario laboral. 1 para turno 1 y 2 para turno 2
df_c1['Horario_laboral'] = df_c1['Hora'].apply(lambda x: 1 if x in range(8, 19) else 0)
# Crear columna para día de la semana
df_c1['Dia_semana'] = df_c1['Fecha'].apply(lambda x: dt.datetime.strptime(x, '%d/%m/%Y').weekday())
#Observamos la base de datos
df_c1.head()

,Fecha,Active_energy,Reactive_energy,Voltaje_FA,Voltaje_FC,Cliente,Sector Económico:,Mes,Año,Dia,Hora,Nombre_dia,Fin_de_semana,Horario_laboral,Dia_semana
0,01/01/2021,0.357841,0.282788,455.139171,510.561002,Cliente 1,Elaboración de cacao y chocolate y de producto...,1,2021,1,0,Friday,0,0,4
1,01/01/2021,0.372264,0.431377,469.978787,469.917178,Cliente 1,Elaboración de cacao y chocolate y de producto...,1,2021,1,1,Friday,0,0,4
2,01/01/2021,1.044687,0.338626,468.721120,546.949147,Cliente 1,Elaboración de cacao y chocolate y de producto...,1,2021,1,2,Friday,0,0,4
3,01/01/2021,0.566425,0.495791,452.329255,444.122989,Cliente 1,Elaboración de cacao y chocolate y de producto...,1,2021,1,3,Friday,0,0,4
4,01/01/2021,1.080556,0.472018,513.477596,535.463719,Cliente 1,Elaboración de cacao y chocolate y de producto...,1,2021,1,4,Friday,0,0,4


In [52]:
# Grafica de correlación entre energia activa y reactiva
fig = px.scatter(df_c1, x='Active_energy', y='Reactive_energy', title='Energia Activa vs Energia Reactiva')
fig.show()

In [55]:
# Crear modelo de KMeans
kmeans = KMeans(n_clusters=2, random_state=0)
# Estandarizar los datos
scaler = StandardScaler()
X = df_c1[['Active_energy', 'Reactive_energy', 'Voltaje_FA', 'Horario_laboral', 'Dia_semana']]
X_scaled = scaler.fit_transform(X)
# Entrenar el modelo
kmeans.fit(X_scaled)
# Predecir los clusters
df_c1['Cluster'] = kmeans.predict(X_scaled)
# Metrocas del modelo
print('Silhouette Score:', silhouette_score(X_scaled, df_c1['Cluster']))
print('Davies Bouldin Score:', davies_bouldin_score(X_scaled, df_c1['Cluster']))
print('Calinski Harabasz Score:', calinski_harabasz_score(X_scaled, df_c1['Cluster']))
# Graficar los clusters
fig = px.scatter(df_c1, x='Fecha', y='Active_energy', color='Cluster', title='Energia Activa vs Energia Reactiva')
fig.show()


/Users/gabrielgomez/Library/Python/3.9/lib/python/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



Silhouette Score: 0.4423018373341092
Davies Bouldin Score: 1.103918861108764
Calinski Harabasz Score: 9318.949357935713


In [48]:
# Crear modelo de DBSCAN
from sklearn.cluster import DBSCAN
# Crear modelo con 2 clusters
dbscan = DBSCAN(eps=0.5, min_samples=5)
# Entrenar modelo
dbscan.fit(X_scaled)
# Predecir los clusters
df_c1['Cluster_DBSCAN'] = dbscan.labels_
# Metrcias del modelo
print('Silhouette Score:', silhouette_score(X_scaled, df_c1['Cluster_DBSCAN']))
print('Davies Bouldin Score:', davies_bouldin_score(X_scaled, df_c1['Cluster_DBSCAN']))
print('Calinski Harabasz Score:', calinski_harabasz_score(X_scaled, df_c1['Cluster_DBSCAN']))
# Graficar los clusters
fig = px.scatter(df_c1, x='Fecha', y='Active_energy', color='Cluster_DBSCAN', title='Energia Activa vs Energia Reactiva')
fig.show()

Silhouette Score: 0.20088122312961051
Davies Bouldin Score: 1.4199445233565782
Calinski Harabasz Score: 2934.109405472223
